In [2]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

## YOLO json

In [44]:
data = pd.read_json("./yolo.json")
confidence = 0.40
dataset = "train" #"test"
dataset_len = 4883 #4871

In [50]:
prediction_strings = []
file_names = []
idx = 0

for i in tqdm(data.groupby(by=["image_id"])):
    image_id = i[0]
    if(idx < image_id): 
        while(idx != image_id):
            prediction_strings.append(" ")
            file_names.append(dataset + "/" + str(idx + 10000)[1:] + ".jpg")
            idx += 1
            
    prediction_string = ""
    for j in i[1].iterrows():
        if j[1].score < confidence:
            continue
        blk = [j[1].category_id, round(j[1].score,15), j[1].bbox[0], j[1].bbox[1], j[1].bbox[0] + j[1].bbox[2], j[1].bbox[1] + j[1].bbox[3]]
        blk = list(map(lambda x : str(x) + " " ,blk))
        for i in blk:
            prediction_string += i
        #print(blk)
        #print(j[1]) 
    file_name = dataset + "/" + str(image_id + 10000)[1:] + ".jpg"
    idx += 1
    prediction_strings.append(prediction_string)
    file_names.append(file_name)

if(idx < dataset_len): 
    while(idx == dataset_len - 1):
        prediction_strings.append(" ")
        file_names.append(dataset + "/" + str(idx + 10000)[1:] + ".jpg")
        idx += 1


100%|██████████| 3906/3906 [00:17<00:00, 218.98it/s]


In [56]:

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./submission.csv', index=None)
submission.head()

,PredictionString,image_id
0,0 0.97872 199.491 193.15 745.552 662.52,train/0000.jpg
1,,train/0001.jpg
2,3 0.97974 265.743 162.129 897.924 675.842,train/0002.jpg
3,2 0.97316 462.929 369.406 695.233 627.20800000...,train/0003.jpg
4,1 0.96775 858.852 412.719 1023.088 613.342 1 0...,train/0004.jpg


---
## Confidence edit  
### COCO json-> CSV

In [7]:
coco = COCO("./coco.json")
confidence = 0.4
#dataset = "test" if coco.loadImgs(coco.getImgIds()[0])[0]['file_name'][:4] == 'test' else "train"

loading annotations into memory...
Done (t=0.93s)
creating index...
index created!


In [8]:
file_names = []
strings = []
for image_id in tqdm(coco.getImgIds()):
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
    string = ""
    for ano in annotation_info_list:
        if ano['score'] < confidence:
            continue
        blk = [ano['category_id'], ano['score'], ano['bbox'][0], ano['bbox'][1], ano['bbox'][2] + ano['bbox'][0], ano['bbox'][3] + ano['bbox'][1]]
        blk = list(map(lambda x : str(x) + " " ,blk))
        for i in blk:
            string += i
    
    file_name = image_info['file_name']
    strings.append(string)
    
    file_names.append(file_name)

100%|██████████| 4871/4871 [00:00<00:00, 18123.74it/s]


In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = strings
submission['image_id'] = file_names
submission.to_csv('./submissionCOCO_040.csv', index=None)
submission.head()

,PredictionString,image_id
0,7 0.998166561126709 219.02938842773438 55.1456...,test/0000.jpg
1,3 0.919633686542511 340.2513122558594 265.1795...,test/0001.jpg
2,1 0.9743971824645996 289.17340087890625 291.05...,test/0002.jpg
3,9 0.98899906873703 166.99288940429688 278.7833...,test/0003.jpg
4,9 0.5510952472686768 443.4324951171875 411.139...,test/0004.jpg
